In [1]:
import pandas as pd
import json
import ast

In [2]:
#Cargamos los datos.
data = []
with open(r'Dataset\australian_users_items.json', 'r', encoding='utf-8') as file:
    for line in file:
        data.append(eval(line))

### Creacion del Dataframe

In [3]:
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [4]:
#Creo un dataframe exploratorio.
df1 = df.iloc[:100]
df1.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [5]:
#Copio el df
df2 = df1.copy()
df2.shape

(100, 5)

In [6]:
#Normalizo la coulmna items que es la que tiene los diccionarios anidados
df_f = pd.json_normalize(df2['items'].explode())

In [7]:
df_f.head(3)

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0


In [8]:
df_f.shape

(10158, 4)

### Despues de la exploracion anterior sacare los datos de las columnas [*item_id*] y [*item_name*]  del df base en un archivo parquet para utilizarlo despues en la api. 

In [9]:
df_items = pd.json_normalize(df['items'].explode())

In [10]:
df_items.head(3)

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0


In [11]:
#Elimino las columnas  'playtime_forever' y ' 'playtime_2weeks' 
df_items = df_items.drop(['playtime_forever', 'playtime_2weeks'], axis=1)

In [12]:
df_items.head(4)

,item_id,item_name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic


In [13]:
df_items.to_csv(r'Dataset\Procesado\id_games.csv', index=False)

In [27]:
df_items.describe()

,item_id,item_name
count,5153209,5153209
unique,10978,10947
top,205790,Dota 2 Test
freq,49571,49571


In [28]:
# Agrupo por 
df_items_sin_duplicados = df_items.drop_duplicates(subset='item_id')

In [29]:
df_items_sin_duplicados.describe()

,item_id,item_name
count,10978,10978
unique,10978,10947
top,10,Dungeon Fighter Online
freq,1,2


### Dataset de *Steam users*

In [4]:
#Creo un nuevo dataframe para trabajar los datos de user id
df_id = df.copy()

In [5]:
df_id.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [6]:
# Elimino del dataset 'user_url' 
df_id =df_id.drop(['user_url'], axis=1)
df_id.head(3)

,user_id,items_count,steam_id,items
0,76561197970982479,277,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [7]:
# Exporto la base de datos para consulta.
df_id.to_csv(r'Dataset\Procesado\user_id_profile.csv')
